In [1]:
import pandas as pd
from pandas_datareader import wb
import country_converter as coco
import os

In [2]:
os.chdir('..')

In [3]:
df = pd.read_pickle(os.path.join('dataframes_pkl', 'df_global_format.pkl'))

In [4]:
list_countries = coco.CountryConverter().convert(df['Area'].unique())

More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression match for China, Taiwan Province of
More then one regular expression m

In [5]:
list_countries = list(map(lambda x: x[1] if type(x) is list else x, list_countries))

In [6]:
gdp_df = wb.download(country = list_countries, indicator = 'NY.GDP.PCAP.CD', start = df['Year'].min(), end = df['Year'].max())

In [7]:
gdp_df[gdp_df['NY.GDP.PCAP.CD'].isnull()]

NY.GDP.PCAP.CD
country          year                
French Polynesia 2017             NaN
                 2016             NaN
                 2015             NaN
                 2014             NaN
Venezuela, RB    2017             NaN
                 2016             NaN
                 2015             NaN

In [8]:
gdp_pyf = wb.download(country = 'PYF', indicator = 'NY.GDP.PCAP.CD', start = 2000, end = 2000).iloc[0,0]

In [9]:
gdp_df.columns = ['GDP_PCAP']

In [10]:
gdp_df.reset_index(inplace = True)

In [11]:
pyf_index = gdp_df[gdp_df['country'] == 'French Polynesia'].index
gdp_df.loc[pyf_index, 'GDP_PCAP'] = gdp_df.loc[pyf_index, 'GDP_PCAP'].fillna(gdp_pyf)

In [12]:
ven_gdp = gdp_df[gdp_df['country'] == 'Venezuela, RB'].iloc[3, 2]

In [13]:
gdp_df.fillna(ven_gdp, inplace = True)

In [14]:
gdp_df.isnull().sum()

country     0
year        0
GDP_PCAP    0
dtype: int64

In [15]:
gdp_df.rename({'country': 'Area', 'year': 'Year'}, axis = 1, inplace = True)

In [16]:
gdp_df.head()

,Area,Year,GDP_PCAP
0,Afghanistan,2017,516.679862
1,Afghanistan,2016,512.012778
2,Afghanistan,2015,556.007221
3,Afghanistan,2014,614.223342
4,Albania,2017,4531.019374


In [17]:
gdp_df.to_pickle(os.path.join('dataframes_pkl', 'df_gdp.pkl'))